# Prior and Posterior predictive distributions

In [8]:
from cmdstanpy import CmdStanModel
import pandas as pd
import arviz as az 
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

### Exercise 1 - Modeling prior predictive distribution

In [10]:
prior_predictive_dist = CmdStanModel(stan_file='code_1.stan')

# Compilation of code_1.stan and get 1000 samples
samples = prior_predictive_dist.sample(fixed_param=True, 
                                       iter_sampling=1000)

# Creation of pandas dataframe from resulting draws
df = samples.draws_pd()
display(df)

# fixed_param=True, iter_sampling=1000, iter_warmup=0, chains=1

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:CmdStan start processing
chain 1 |██████████| 00:00 Sampling completed


INFO:cmdstanpy:CmdStan done processing.


,lp__,accept_stat__,N,y,p
0,0.0,0.0,50.0,13.0,0.245822
1,0.0,0.0,50.0,5.0,0.084340
2,0.0,0.0,50.0,14.0,0.297394
3,0.0,0.0,50.0,12.0,0.237646
4,0.0,0.0,50.0,15.0,0.295046
...,...,...,...,...,...
995,0.0,0.0,50.0,17.0,0.329834
996,0.0,0.0,50.0,9.0,0.137905
997,0.0,0.0,50.0,6.0,0.119000
998,0.0,0.0,50.0,7.0,0.220732
